# Скачиваем данные и устанавливаем программы

In [ ]:
%%capture
!wget https://raw.githubusercontent.com/SpryGorgon/hse22_hw2/main/data/gms2.lst
!wget https://raw.githubusercontent.com/SpryGorgon/hse22_hw2/main/data/scaffolds.fasta
!wget https://raw.githubusercontent.com/SpryGorgon/hse22_hw2/main/data/proteins.fasta
!wget https://raw.githubusercontent.com/SpryGorgon/hse22_hw2/main/data/scaffolds.hits_from_MIL_1.txt
!wget https://raw.githubusercontent.com/SpryGorgon/hse22_hw2/main/data/scaffolds.hits_from_SwissProt.txt

## Скачиваем геном близкородственной бактерии T.oleivorans

Геном, последовательности генов (нт) и белков (протеом) для бактерии Thalassolituus oleivorans MIL-1
https://www.ncbi.nlm.nih.gov/nuccore/HF680312

In [ ]:
!sh -c "$(curl -fsSL ftp://ftp.ncbi.nlm.nih.gov/entrez/entrezdirect/install-edirect.sh)"


Entrez Direct has been successfully downloaded and installed.

In order to complete the configuration process, please execute the following:

  echo "export PATH=\${PATH}:/root/edirect" >> ${HOME}/.bashrc

or manually edit the PATH variable assignment in your .bashrc file.

Would you like to do that automatically now? [y/N]
y
OK, done.

To activate EDirect for this terminal session, please execute the following:

export PATH=${PATH}:${HOME}/edirect



In [ ]:
!$HOME/edirect/efetch -db nuccore -id HF680312 -format gb  >  T_oleivorans_MIL_1.gbk

In [ ]:
%%capture
!pip install biopython

## Аннотация генома

Biopython Tutorial: http://biopython.org/DIST/docs/tutorial/Tutorial.html

In [ ]:
!head -n 20 gms2.lst

# GeneMark.hmm-2 LST format
# GeneMark.hmm-2 prokaryotic version: 1.25_lic
# File with sequence: scaffolds.fasta
# File with native parameters: GMS2.mod
# Native species name and build: unspecified GeneMarkS-2-1.14_1.25_lic
# File with MetaGeneMark parameters: /content/gms2_linux_64/mgm_11.mod
# translation table: 11
# output date start: Wed Oct 12 19:41:49 2022

# sequence-region 1 3880139
SequenceID: scaffold1_cov232
     1   -   <3    299     297 native GAGGTG 4 1
     2   -    372    815     444 atypical AGCGAG 6 1
     3   -    840    1361     522 atypical CAGGAA 7 1
     4   -    1799    2626     828 atypical CAGGAT 10 1
     5   +    2972    3094     123 native TTGGAG 7 1
     6   -    3269    4003     735 native CAGGTC 8 1
     7   -    4341    5516    1176 native TAGGAG 9 1
     8   -    5541    7688    2148 native GTGGAG 8 1
     9   +    8057    8851     795 native AAGTAG 6 1


In [ ]:
from Bio import SeqIO

records={}
for record in SeqIO.parse("/content/scaffolds.fasta", "fasta"):
  records[record.id] = record
  records[record.id].annotations['molecule_type'] = 'DNA'
  records[record.id].format("genbank")

In [ ]:
SeqIO.write([record[1] for record in records.items()], "GENOME.gbk", "genbank")

69

## Добавляем координаты генов

```
SequenceID: scaffold3_cov275
  3535   -    30    1001     972 atypical GTCGAG 5 1
  3536   -    1350    2192     843 native AAAGTG 7 1
  3537   +    2304    2507     204 native GCGGAG 7 1
  3538   -    2554    2751     198 native TAAGTA 7 1
  3539   +    2855    3265     411 native TCGGTC 6 1
```

In [ ]:
from pprint import pprint
from Bio.SeqFeature import SeqFeature, FeatureLocation

genes = {}

skip=9
with open("gms2.lst") as f:
    lines = f.readlines()
    lines = [line.strip() for line in lines[skip:]]
    reading=False
    buf=[]
    name=''
    for line in lines:
        if(len(line)==0):
            continue
        if(line[0]=='#'):
            if reading:
                if(len(buf)>0):
                    genes[name] = buf.copy()
                reading=False
                name=''
                buf=[]
            else:
                reading=True
            continue
        if(name==''):
            name = line.split()[1]
            continue
        buf.append(line.split())

In [ ]:
for key in records:
    record = records[key]
    features=[]
    if record.name not in genes:
        continue
    for gene in genes[record.name]:
        feature = SeqFeature(FeatureLocation(int(str(gene[2]).strip('<>')), int(str(gene[3]).strip('<>')), (1 if gene[1]=='+' else -1)), type='CDS')
        feature.qualifiers['locus_tag'] = [gene[0]]
        features.append(feature)
    records[key].features = features.copy()
pprint([record[1] for record in records.items()][0].features[5].qualifiers)

OrderedDict([('locus_tag', ['6'])])


In [ ]:
SeqIO.write([record[1] for record in records.items()], "GENOME.gbk", "genbank")

69

## Добавляем трансляции генов (аминокислотные посл-ти)

In [ ]:
for protein in SeqIO.parse("proteins.fasta", "fasta"):
    for key in records:
        for feature in records[key].features:
            if protein.id == feature.qualifiers['locus_tag'][0]:
                feature.qualifiers['translation'] = [protein.seq]
pprint([record[1] for record in records.items()][0].features[5].qualifiers)

OrderedDict([('locus_tag', ['6']),
             ('translation',
              [Seq('VDLTLDFDLSIGEIKYTDDDVFDGGSLSVSGVTLGGGAGRTTLFGEAVTNTSRI...AGH')])])


In [ ]:
SeqIO.write([record[1] for record in records.items()], "GENOME.gbk", "genbank")

69

## Добавляем функции белков (из бактерии MIL-1)

Часть файла scaffolds.hits_from_MIL_1.txt:
```
3535	lcl|HF680312.1_prot_CCU72326.1_1877	99.054	317	3	0	3	319	1	317	0.0	644
3535	lcl|HF680312.1_prot_CCU72392.1_1943	34.583	240	138	5	85	307	956	1193	2.33e-30	118
3535	lcl|HF680312.1_prot_CCU72759.1_2310	27.667	300	188	8	8	302	709	984	1.02e-24	101
3535	lcl|HF680312.1_prot_CCU70543.1_94	42.857	126	71	1	79	204	798	922	5.63e-24	99.4
3535	lcl|HF680312.1_prot_CCU70690.1_241	26.140	329	205	10	2	303	591	908	6.26e-23	96.7
3535	lcl|HF680312.1_prot_CCU74013.1_3564	38.462	130	76	3	91	219	685	811	4.96e-20	87.8
3535	lcl|HF680312.1_prot_CCU71475.1_1026	36.522	115	71	2	86	200	555	667	2.19e-16	76.6
3535	lcl|HF680312.1_prot_CCU71362.1_913	24.535	269	187	3	52	306	633	899	2.61e-14	70.5
3535	lcl|HF680312.1_prot_CCU70724.1_275	30.534	131	81	3	78	205	1983	2106	6.54e-13	66.2
3535	lcl|HF680312.1_prot_CCU70719.1_270	26.087	115	83	1	90	204	15	127	9.14e-12	58.5
3535	lcl|HF680312.1_prot_CCU72006.1_1557	35.652	115	68	3	87	200	421	530	1.06e-11	62.0
3535	lcl|HF680312.1_prot_CCU72513.1_2064	33.628	113	72	1	91	200	766	878	1.22e-11	62.4
3536	lcl|HF680312.1_prot_CCU72327.1_1878	100.000	280	0	0	1	280	1	280	0.0	570
3536	lcl|HF680312.1_prot_CCU72858.1_2409	27.437	277	181	4	10	272	1	271	1.82e-26	101
3537	lcl|HF680312.1_prot_CCU72328.1_1879	100.000	67	0	0	1	67	1	67	9.75e-44	132
```

In [ ]:
!head scaffolds.hits_from_MIL_1.txt

1	lcl|HF680312.1_prot_CCU71653.1_1204	98.990	99	1	0	1	99	1	99	8.08e-68	196
1	lcl|HF680312.1_prot_CCU72283.1_1834	97.980	99	2	0	1	99	1	99	2.29e-67	194
1	lcl|HF680312.1_prot_CCU71933.1_1484	97.980	99	2	0	1	99	1	99	4.15e-66	191
1	lcl|HF680312.1_prot_CCU71592.1_1143	97.980	99	2	0	1	99	1	99	4.15e-66	191
1	lcl|HF680312.1_prot_CCU71865.1_1416	96.774	62	2	0	1	62	1	62	2.15e-42	130
1	lcl|HF680312.1_prot_CCU71866.1_1417	95.349	43	2	0	1	43	1	43	4.93e-27	90.9
1	lcl|HF680312.1_prot_CCU71222.1_773	100.000	46	0	0	54	99	7	52	2.12e-24	84.7
3	lcl|HF680312.1_prot_CCU71928.1_1479	100.000	173	0	0	1	173	1	173	1.66e-130	359
5	lcl|HF680312.1_prot_CCU71924.1_1475	97.143	35	1	0	1	35	1	35	1.51e-19	75.1
6	lcl|HF680312.1_prot_CCU71587.1_1138	98.770	244	3	0	1	244	38	281	2.17e-171	470


In [ ]:
functions = {}
with open('scaffolds.hits_from_MIL_1.txt', 'r') as f:
    lines = f.readlines()
    prev = ''
    for line in lines:
        ls = line.split()
        if ls[0]!=prev:
            functions[ls[0]] = ls[1].split('_')[2]
            prev=ls[0]

for record_key in records:
    for i,feature in enumerate(records[record_key].features):
        tag = feature.qualifiers['locus_tag'][0]
        if tag in functions.keys():
            records[record_key].features[i].qualifiers['note'] = [functions[tag]]
pprint([record[1] for record in records.items()][0].features[5].qualifiers)

OrderedDict([('locus_tag', ['6']),
             ('translation',
              [Seq('VDLTLDFDLSIGEIKYTDDDVFDGGSLSVSGVTLGGGAGRTTLFGEAVTNTSRI...AGH')]),
             ('note', ['CCU71587.1'])])


In [ ]:
mil_1_genome = SeqIO.read("/content/T_oleivorans_MIL_1.gbk", "genbank")

In [ ]:
for  mil_f in mil_1_genome.features:
    if 'protein_id' not in mil_f.qualifiers:
        continue

    for key, record in records.items():
        for i,feature in enumerate(record.features):
            if 'note' not in feature.qualifiers:
                continue
            if feature.qualifiers['note'][0] == mil_f.qualifiers['protein_id'][0]:
                records[key].features[i].qualifiers['product'] = mil_f.qualifiers['product']
pprint([record[1] for record in records.items()][0].features[5].qualifiers)

OrderedDict([('locus_tag', ['6']),
             ('translation',
              [Seq('VDLTLDFDLSIGEIKYTDDDVFDGGSLSVSGVTLGGGAGRTTLFGEAVTNTSRI...AGH')]),
             ('note', ['CCU71587.1']),
             ('product', ['hypothetical protein'])])


In [ ]:
SeqIO.write([record[1] for record in records.items()], "GENOME.gbk", "genbank")

69

## Добавляем функции белков (из БД SwissProt)

Читаем информацию из файла scaffolds.hits_from_SwissProt.txt

In [ ]:
!head scaffolds.hits_from_SwissProt.txt

169	sp|P55546|Y4LF_SINFN	42.553	141	80	1	1	141	1	140	1.17e-36	144
428	sp|Q06916|GUFA_MYXXA	43.802	242	136	0	71	312	13	254	2.33e-47	162
428	sp|Q3B4G1|ZUPT_CHLL3	34.016	244	148	6	72	304	17	258	6.82e-25	103
428	sp|Q8N1S5|S39AB_HUMAN	40.625	160	83	3	162	312	186	342	1.64e-23	101
428	sp|Q6P6S2|S39AB_RAT	40.000	160	84	3	162	312	179	335	2.03e-23	101
428	sp|Q2YDD4|S39AB_BOVIN	40.000	160	84	3	162	312	185	341	2.62e-23	101
428	sp|Q8NQK0|ZUPT_CORGL	35.968	253	144	7	67	306	10	257	1.83e-22	97.4
428	sp|Q8BWY7|S39AB_MOUSE	41.176	153	78	3	169	312	193	342	2.70e-22	98.6
428	sp|B3ECE6|ZUPT_CHLL2	34.000	250	146	10	71	306	16	260	5.36e-22	96.3
428	sp|Q28J44|S39AB_XENTR	39.623	159	84	3	163	312	181	336	1.08e-21	96.7


Получить функции белков SwissProt из файла: https://ftp.uniprot.org/pub/databases/uniprot/current_release/knowledgebase/complete/uniprot_sprot.dat.gz

In [ ]:
%%capture
!wget https://ftp.uniprot.org/pub/databases/uniprot/current_release/knowledgebase/complete/uniprot_sprot.dat.gz

In [ ]:
!gzip -d uniprot_sprot.dat.gz

In [ ]:
uniprot = SeqIO.index("/content/uniprot_sprot.dat", "swiss")

In [ ]:
swiss_notes={}
with open('scaffolds.hits_from_SwissProt.txt', 'r') as f:
    lines = f.readlines()
    prev = ''
    for line in lines:
        ls = line.split()
        if ls[0]!=prev:
            swiss_notes[ls[0]] = ls[1].split('|')[1]
            prev=ls[0]

In [ ]:
for key, record in records.items():
    for i,feature in enumerate(record.features):
        tag = feature.qualifiers['locus_tag'][0]
        if tag not in swiss_notes.keys():
            continue
        
        try:
            records[key].features[i].qualifiers['note'] = [swiss_notes[tag]]
            records[key].features[i].qualifiers['product'] = uniprot[swiss_notes[tag]].features[0].qualifiers['note']
        except:
            pass
pprint([record[1] for record in records.items()][0].features[168].qualifiers)

OrderedDict([('locus_tag', ['169']),
             ('translation',
              [Seq('MPTKIFISYSHKDEEFKNSLAEHLAGLERSGAISEWNDRKIAPGTDWSHEINEN...RIG')]),
             ('note', ['P55546']),
             ('product', 'Uncharacterized protein y4lF')])


In [ ]:
SeqIO.write([record[1] for record in records.items()], "GENOME.gbk", "genbank")

69